<a href="https://colab.research.google.com/github/canhphu/ML/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-09-06 15:37:54--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  70.6MB/s    in 0.4s    

2024-09-06 15:37:54 (70.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [5]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
# add your code here - consider creating a new cell for each section of code
# Step 1: Handle missing authors
df_books.dropna(subset=['author'], inplace=True)

# Step 2: Filter users with at least 200 ratings
user_ratings_count = df_ratings['user'].value_counts()
valid_users = user_ratings_count[user_ratings_count >= 200].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(valid_users)]

# Step 3: Filter books with at least 100 ratings
book_ratings_count = df_ratings['isbn'].value_counts()
valid_books = book_ratings_count[book_ratings_count >= 100].index
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(valid_books)]
print("Number of unique users:", df_ratings_filtered['user'].count())
print("Number of unique books:", df_ratings_filtered['isbn'].count())
print("Number of unique ratings:", df_ratings_filtered.shape[0])
# Check if the filtered DataFrame is empty
if df_ratings_filtered.empty:
    print("Error: No common books found after filtering.")
else:
    # Step 5: Create the book-user matrix
    book_user_matrix = df_ratings_filtered.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

    # Step 6: Train the NearestNeighbors model
    from sklearn.neighbors import NearestNeighbors
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
    model_knn.fit(book_user_matrix)
    print("Model fitted successfully.")

Number of unique users: 49781
Number of unique books: 49781
Number of unique ratings: 49781
Model fitted successfully.


In [7]:
# function to return recommended books - this will be tested
def get_recommends(book_title=""):
    # Step 1: Find the ISBN for the given book title
    if book_title not in df_books['title'].values:
        return f"Book '{book_title}' not found in the dataset."

    # Get the ISBN of the input book
    book_isbn = df_books[df_books['title'] == book_title]['isbn'].values[0]

    # Step 2: Find the index of the book in the book-user matrix
    if book_isbn not in book_user_matrix.index:
        return f"Book '{book_title}' does not have enough ratings for recommendations."

    book_index = book_user_matrix.index.get_loc(book_isbn)

    # Step 3: Use the KNN model to find nearest neighbors (including itself)
    distances, indices = model_knn.kneighbors(book_user_matrix.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    # Step 4: Retrieve the recommended books and their distances
    recommended_books = []
    for i in range(1, len(distances.flatten())):  # Skip the first one as it is the input book itself
        similar_book_isbn = book_user_matrix.index[indices.flatten()[i]]
        similar_book_title = df_books[df_books['isbn'] == similar_book_isbn]['title'].values[0]
        recommended_books.append([similar_book_title, distances.flatten()[i]])
    # Step 6: Sort recommended books by distance in descending order (nearest first)
    recommended_books.sort(key=lambda x: x[1], reverse=True)  # Sort by distance in descending order
    recommended_books = recommended_books[:5]  # Keep only the top 5 recommendations

    # Return the input book and the recommended books
    return [book_title, recommended_books]

# Testing the function
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))


['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Catch 22', 0.7939835], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Interview with the Vampire', 0.73450685], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]


In [8]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
